In [10]:
import pandas as pd
import pickle
import numpy as np

In [3]:
pandas_ds = pickle.load(open( "pandas_processed_topics.p", "rb" ))

In [4]:
topic_frequency = pandas_ds.groupby('topic')['body'].count()

In [38]:
MentorTopic = pandas_ds.groupby(['from', 'topic'])['body'].count()

In [105]:
flatMentorTopic = pd.DataFrame(MentorTopic)
flatMentorTopic.reset_index(level=1, inplace=True)
flatMentorTopic.reset_index(level=0, inplace=True)
flatMentorTopic = flatMentorTopic.rename(columns={'body': 'count'})

In [147]:
MentorTopicMaxCount = pandas_ds.groupby(['from', 'topic'])['body'].count().groupby('topic').max()
MentorTopicMaxCount

topic
0    2221
1     724
2    5329
3     116
4    2168
5    2021
6     195
7     232
8    3776
9     113
Name: body, dtype: int64

In [150]:
def fdiv(x, y):
    fcount = topic_frequency[y]
    return float(x) / fcount
#flatMentorTopic.head()['count']

def ydiv(x,y):
    ycount = MentorTopicMaxCount[y]
    return float(x) / ycount

flatMentorTopic['share'] = flatMentorTopic.apply(lambda row: fdiv(row['count'], row['topic']), axis=1)
flatMentorTopic['normalized'] = flatMentorTopic.apply(lambda row: ydiv(row['count'], row['topic']), axis=1)

In [ ]:
flatMentorTopic.sort_values(['share'], ascending=[0]).head()

In [135]:
newflatMentorTopic = flatMentorTopic
newflatMentorTopicPivot = newflatMentorTopic.pivot(index='from', columns='topic', values='share').fillna(0)

In [136]:
newflatMentorTopicPivot.head()

topic,0,1,2,3,4,5,6,7,8,9
from,,,,,,,,,,
'todd'.delahoussaye@enron.com,0.00000,0.0,0.00032,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
--migrated--bmishkin@ercot.com,0.00001,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
--migrated--dodle@ercot.com,0.00001,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
-nikole@excite.com,0.00001,0.0,0.00000,0.0,0.000104,0.000013,0.0,0.0,0.0,0.0
-persson@ricemail.ricefinancial.com,0.00000,0.0,0.00000,0.0,0.000086,0.000000,0.0,0.0,0.0,0.0


In [165]:
newYflatMentorTopic = flatMentorTopic
newYflatMentorTopicPivot = newflatMentorTopic.pivot(index='from', columns='topic', values='normalized').fillna(0)

In [153]:
newYflatMentorTopicPivot.head()

topic,0,1,2,3,4,5,6,7,8,9
from,,,,,,,,,,
'todd'.delahoussaye@enron.com,0.00000,0.0,0.000938,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
--migrated--bmishkin@ercot.com,0.00045,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
--migrated--dodle@ercot.com,0.00045,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
-nikole@excite.com,0.00045,0.0,0.000000,0.0,0.002768,0.000495,0.0,0.0,0.0,0.0
-persson@ricemail.ricefinancial.com,0.00000,0.0,0.000000,0.0,0.002306,0.000000,0.0,0.0,0.0,0.0


In [191]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(newYflatMentorTopicPivot)

In [203]:
def getRecommendations(knn, input_array):
    distances, indices = knn.kneighbors(input_array)
    output_recommendations = []
    for i in indices[0]:
        name = newYflatMentorTopicPivot.ix[i].name
        output_recommendations.append(name)
    return output_recommendations

In [223]:
"""
    Inputs is based on a desired level of expertise across topic
    the input needs to be provised between 0 and 1
"""
input_array = np.array([0,1,0,0,0, 0,0,0,0,0]).reshape(1, -1)
getRecommendations(knn, input_array)

['outlook.team@enron.com',
 'stacey.white@enron.com',
 'holly.keiser@enron.com',
 'joanne.rozycki@enron.com',
 'patti.thompson@enron.com']